In [ ]:

import CodaPCA

import numpy as np

from runpca import read_csv

import os

import sklearn

from sklearn.linear_model import Ridge

#change module for newer sklearn versions

from sklearn.cross_validation  import cross_val_score

from sklearn.cross_validation  import KFold

import matplotlib.pyplot as plt

#read in the data. Given an array of which files are regression, classification or unlabelled

data_path = os.getcwd() + "\\Aitchinson"


regression_list = [3,4,5,18,21,34,39]

classification_list = [7,8,9,11,12,16,17,19,23,24,25,26,28,29,33,37]

unlabelled_list=[1,2,6,10,13,14,15,20,22,27,30,31,32,35,36,38,40]



r_files = []

c_files = []

u_files = []




for file in os.listdir(data_path):

    for i in regression_list:

        if os.path.isfile(os.path.join(data_path,file)) and 'Data ' + str(i) + '.' in file:

            r_files.append("Aitchinson/" + file)

    for i in classification_list:

        if os.path.isfile(os.path.join(data_path,file)) and 'Data ' + str(i) + '.' in file:

            c_files.append("Aitchinson/" + file)

    for i in unlabelled_list:

        if os.path.isfile(os.path.join(data_path,file)) and 'Data ' + str(i) + '.' in file:

            u_files.append("Aitchinson/" + file)



#need to specify where the the targets and features are in the dataset, and whether there are non compositional features



def PCA_Regression(data, co_feature_indices, target_index, 

                   other_feature_indices = [], alg=CodaPCA.Alg.CODAPCA, verbose=False):

    

    #can loop through/optimise this in another way?

    

    headers = data[1]

    features = data[0][:,co_feature_indices]

    targets = data[0][:,target_index]

    

    #normalise the compositional features. TODO anything extra to deal with non compositional features?

    features = np.array([feat/sum(feat) for feat in features])



    #can be empty

    extra_features = data[0][:,other_feature_indices]

    

    #TODO double check this

    features = np.hstack([features, extra_features])

    

    #compute the CoDA-PCA projection 

    #TODO add component number as a hyperparameter to optimise 

    n_components=len(co_feature_indices)-2



    pca = CodaPCA.CodaPCA(n_components,lrate=1e-3,nn_shape=[100,100], alg=alg)

    pca.fit(features)

    

    Y_coda = pca.project(features)



    pca_clr = CodaPCA.CLRPCA(n_components)

    pca_clr.fit(features)

    

    Y_clr = pca_clr.project(features)


    lm = Ridge()

    #exp the projection to get out of clr space

    coda_score = enhanced_cross_val(lm,np.exp(Y_coda), targets)

    clr_score = enhanced_cross_val(lm,np.exp(Y_clr), targets) 

    naive_score = enhanced_cross_val(lm, features, targets)





    if verbose:

        print("CoDA-PCA:")

        print(coda_score)

        print("CLR-PCA:")

        print(clr_score)

        print ("Naive regression:")

        print (naive_score)

    



    return coda_score,clr_score,naive_score



#training methodology as described in:

#https://papers.nips.cc/paper/3215-learning-with-transformation-invariant-kernels.pdf

def enhanced_cross_val(model, features, targets):

    assert len(features) == len(targets), "Mismatch in length of features and targets"

    

    #define the number of splits and folds uised in the parameter selection

    #stick to smaller splits since we have small datasets

    splits = 4

    param_splits = 3

    

    #split data 

    kf = KFold(len(features), n_folds=splits)

    kfold_scores = []

    for train, test in kf:        

        Y_train = targets[train]

        X_train = features[train]

        

       

        Y_test = targets[test]

        X_test = features[test]

        

        #inner loop for parameter selection (regularisation term in Ridge Regression):

        param_grid = [0.01,0.05,0.1,0.5,1.0,2.0,5.0,10.0,20.0,100.0]

        for a in param_grid:

            max_score = -np.inf

            lm = Ridge(a)

   

            curr_score = np.mean(cross_val_score(lm, X_train, Y_train,cv=param_splits))

            if curr_score > max_score:

                max_score = curr_score

                best_param = a

        

        #compute test score based on best parameter

        lm = Ridge(best_param)

        lm.fit(X_train, Y_train)

        kfold_scores.append(lm.score(X_test, Y_test))

        

    return kfold_scores







#can automate this if we had assume a certain structure for the indices of features and targets, or an array per dataset 



score_dict = {}


data_18_scores = PCA_Regression(read_csv(r_files[0], normalize=False), co_feature_indices=[0,1,2,3], target_index=4)

#other "target" also at 5

score_dict['18'] = data_18_scores



data_21_scores = PCA_Regression(read_csv(r_files[1], normalize=False), co_feature_indices=[0,1,2,3], target_index=4) 

score_dict['21'] = data_21_scores



data_3_scores = PCA_Regression(read_csv(r_files[2], normalize=False), co_feature_indices=[0,1,2,3,4], target_index=5) 

score_dict['3'] = data_3_scores



data_34_scores = PCA_Regression(read_csv(r_files[3], normalize=False), co_feature_indices=[0,1,2,3], target_index=4)

score_dict['34'] = data_34_scores



data_39_scores = PCA_Regression(read_csv(r_files[4], normalize=False), co_feature_indices=[0,1,2], target_index=3)

score_dict['39'] = data_39_scores



data_4_scores = PCA_Regression(read_csv(r_files[5], normalize=False), co_feature_indices=[0,1,2,3,4], target_index=5)

score_dict['4'] = data_4_scores



data_5_scores =PCA_Regression(read_csv(r_files[6], normalize=False), co_feature_indices=[0,1,2], target_index=3)

score_dict['5'] = data_5_scores



#note: plotly code works fine, but gives a jupyter warning when saving with a rendered table



    



